In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Bionic AI/')
from functions import *

## Get All Data

In [ ]:
allData, allEvents = getAllData()

reading subject 1 out of 12
reading subject 2 out of 12
reading subject 3 out of 12
reading subject 4 out of 12
reading subject 5 out of 12
reading subject 6 out of 12
reading subject 7 out of 12
reading subject 8 out of 12
reading subject 9 out of 12
reading subject 10 out of 12
reading subject 11 out of 12
reading subject 12 out of 12
concatenating all data


In [ ]:
allData.shape

(17977414, 32)

## Get Individual Subjects' Data

In [ ]:
subjects, events = getSubjectsData(test_series=3,Test=False)

subjects_test, events_test = getSubjectsData(test_series=3,Test=True)

#**Scenario 1: Train on Individual Subjects, Predict for Individual Subjects**

## Scenario 1: Base Model

In [ ]:
LR = LR().double()

In [ ]:
optimizer = torch.optim.Adadelta(LR.parameters(), lr=1, eps=1e-10) # wtf is epsilon???

In [ ]:
for i in range(12):
    print(f'training subject {i+1} out of 12')
    for layer in LR.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    optimizer = torch.optim.Adadelta(LR.parameters(), lr=1, eps=1e-10)
    train(LR,subjects[i],events[i],1,2000)
    torch.save(LR.state_dict(), f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/Base/Subject{i+1}_Model")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
allAucs = {}
allPreds = {}
allTests = {}
for i in range(12):
  print(f'predicting for subject {i+1} out of 12')
  for layer in LR.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
  LR.load_state_dict(torch.load(f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/Base/Subject{i+1}_Model"))
  preds, test = getPredictions(LR,subjects_test[i],events_test[i],512,2000)
  aucs = [auc(test[:,j], preds[:,j]) for j in range(6)]
  allAucs[f'subject{i+1}'] = aucs
  allPreds[f'subject{i+1}'] = preds
  allTests[f'subject{i+1}'] = test

predicting for subject 1 out of 12
predicting for subject 2 out of 12
predicting for subject 3 out of 12
predicting for subject 4 out of 12
predicting for subject 5 out of 12
predicting for subject 6 out of 12
predicting for subject 7 out of 12
predicting for subject 8 out of 12
predicting for subject 9 out of 12
predicting for subject 10 out of 12
predicting for subject 11 out of 12
predicting for subject 12 out of 12


In [ ]:
# save results

AUC = pd.DataFrame.from_dict(allAucs,orient='index')
AUC['average AUC'] = AUC.mean(axis=1)
AUC.from_dict(allAucs,orient='index').to_csv('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/Base/AUC',index=False)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/Base/allPreds', 'wb') as f:
    pickle.dump(allPreds, f)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/Base/allTests', 'wb') as f:
    pickle.dump(allTests, f)

In [ ]:
AUC

,0,1,2,3,4,5,average AUC
subject1,0.579299,0.663222,0.690790,0.704497,0.748452,0.750391,0.689442
subject2,0.828143,0.709157,0.721795,0.735060,0.892268,0.796950,0.780562
subject3,0.717285,0.701068,0.679931,0.638718,0.663618,0.644264,0.674147
subject4,0.692656,0.830885,0.797295,0.797425,0.536241,0.540481,0.699164
subject5,0.648695,0.688881,0.701100,0.718931,0.631891,0.624045,0.668924
subject6,0.589511,0.699851,0.744050,0.722266,0.731672,0.737547,0.704149
subject7,0.833396,0.840221,0.794052,0.848427,0.811392,0.793245,0.820122
subject8,0.652044,0.700322,0.711098,0.687671,0.569203,0.572579,0.648819
subject9,0.597546,0.743383,0.744297,0.761969,0.745392,0.745275,0.722977
subject10,0.708990,0.702512,0.726167,0.682886,0.761975,0.746550,0.721513


# Scenario 1: CNN Model


In [ ]:
CNN = CNN().double()

In [ ]:
optimizer = torch.optim.Adadelta(CNN.parameters(), lr=1, eps=1e-10) # wtf is epsilon???

In [ ]:
for i in range(12):
    print(f'training subject {i+1} out of 12')
    for layer in CNN.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    optimizer = torch.optim.Adadelta(CNN.parameters(), lr=1, eps=1e-10)
    train(CNN,subjects[i],events[i],1,2000)
    torch.save(CNN.state_dict(), f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/CNN/Subject{i+1}_Model")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
optimizer = torch.optim.Adadelta(CNN.parameters(), lr=1, eps=1e-10) # wtf is epsilon???

In [ ]:
allAucs = {}
allPreds = {}
allTests = {}
for i in range(12):
  print(f'predicting for subject {i+1} out of 12')
  for layer in CNN.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
  CNN.load_state_dict(torch.load(f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/CNN/Subject{i+1}_Model"))
  preds, test = getPredictions(CNN,subjects_test[i],events_test[i],512,2000)
  aucs = [auc(test[:,j], preds[:,j]) for j in range(6)]
  allAucs[f'subject{i+1}'] = aucs
  allPreds[f'subject{i+1}'] = preds
  allTests[f'subject{i+1}'] = test

predicting for subject 1 out of 12
predicting for subject 2 out of 12
predicting for subject 3 out of 12
predicting for subject 4 out of 12
predicting for subject 5 out of 12
predicting for subject 6 out of 12
predicting for subject 7 out of 12
predicting for subject 8 out of 12
predicting for subject 9 out of 12
predicting for subject 10 out of 12
predicting for subject 11 out of 12
predicting for subject 12 out of 12


In [ ]:
# save results

AUC = pd.DataFrame.from_dict(allAucs,orient='index')
AUC['average AUC'] = AUC.mean(axis=1)
AUC.from_dict(allAucs,orient='index').to_csv('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/CNN/AUC',index=False)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/CNN/allPreds', 'wb') as f:
    pickle.dump(allPreds, f)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 1 Models/CNN/allTests', 'wb') as f:
    pickle.dump(allTests, f)

In [ ]:
AUC

,0,1,2,3,4,5,average AUC
subject1,0.785809,0.978453,0.959230,0.900385,0.922989,0.904833,0.908617
subject2,0.892107,0.925300,0.911646,0.738754,0.945161,0.870577,0.880591
subject3,0.771831,0.797862,0.783938,0.751164,0.754767,0.785124,0.774114
subject4,0.779742,0.970818,0.968065,0.923953,0.697888,0.651893,0.832060
subject5,0.822514,0.867288,0.856462,0.808551,0.820150,0.807101,0.830344
subject6,0.849034,0.958392,0.945469,0.898689,0.798002,0.828795,0.879730
subject7,0.905102,0.952953,0.941642,0.928115,0.950204,0.910384,0.931400
subject8,0.649692,0.832204,0.814245,0.720120,0.866659,0.804707,0.781271
subject9,0.826171,0.916384,0.907850,0.872554,0.885273,0.886567,0.882467
subject10,0.819930,0.907601,0.900380,0.880461,0.921564,0.844753,0.879115


# **Scenario 2: Train on All Data, Predict for Individual Subjects**

##Scenario 2: Base Model

In [ ]:
train(LR,allData,allEvents,1,2000)
torch.save(LR.state_dict(), "/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/Base/All_Subjects_Model")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
allAucs = {}
allPreds = {}
allTests = {}
for i in range(12):
  print(f'predicting for subject {i+1} out of 12')
  for layer in LR.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
  LR.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/Base/All_Subjects_Model"))
  preds, test = getPredictions(LR,subjects_test[i],events_test[i],512,2000)
  aucs = [auc(test[:,j], preds[:,j]) for j in range(6)]
  allAucs[f'subject{i+1}'] = aucs
  allPreds[f'subject{i+1}'] = preds
  allTests[f'subject{i+1}'] = test

predicting for subject 1 out of 12
predicting for subject 2 out of 12
predicting for subject 3 out of 12
predicting for subject 4 out of 12
predicting for subject 5 out of 12
predicting for subject 6 out of 12
predicting for subject 7 out of 12
predicting for subject 8 out of 12
predicting for subject 9 out of 12
predicting for subject 10 out of 12
predicting for subject 11 out of 12
predicting for subject 12 out of 12


In [ ]:
# save results

AUC2 = pd.DataFrame.from_dict(allAucs,orient='index')
AUC2['average AUC'] = AUC2.mean(axis=1)
AUC2.from_dict(allAucs,orient='index').to_csv('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/Base/AUC2',index=False)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/Base/allPreds2', 'wb') as f:
    pickle.dump(allPreds, f)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/Base/allTests2', 'wb') as f:
    pickle.dump(allTests, f)

## Scenario 2: CNN Model

In [ ]:
CNN = CNN().double()

In [ ]:
optimizer = torch.optim.Adadelta(CNN.parameters(), lr=1, eps=1e-10) # wtf is epsilon???

In [ ]:
train(CNN,allData,allEvents,1,2000)
torch.save(CNN.state_dict(), "/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/CNN/All_Subjects_Model")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:

allAucs = {}
allPreds = {}
allTests = {}
for i in range(12):
  print(f'predicting for subject {i+1} out of 12')
  for layer in CNN.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
  CNN.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/CNN/All_Subjects_Model"))
  preds, test = getPredictions(CNN,subjects_test[i],events_test[i],512,2000)
  aucs = [auc(test[:,j], preds[:,j]) for j in range(6)]
  allAucs[f'subject{i+1}'] = aucs
  allPreds[f'subject{i+1}'] = preds
  allTests[f'subject{i+1}'] = test

predicting for subject 1 out of 12
predicting for subject 2 out of 12
predicting for subject 3 out of 12
predicting for subject 4 out of 12
predicting for subject 5 out of 12
predicting for subject 6 out of 12
predicting for subject 7 out of 12
predicting for subject 8 out of 12
predicting for subject 9 out of 12
predicting for subject 10 out of 12
predicting for subject 11 out of 12
predicting for subject 12 out of 12


In [ ]:
# save results

AUC2 = pd.DataFrame.from_dict(allAucs,orient='index')
AUC2['average AUC'] = AUC2.mean(axis=1)
AUC2.from_dict(allAucs,orient='index').to_csv('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/CNN/AUC2',index=False)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/CNN/allPreds2', 'wb') as f:
    pickle.dump(allPreds, f)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/CNN/allTests2', 'wb') as f:
    pickle.dump(allTests, f)

In [ ]:
AUC2

,0,1,2,3,4,5,average AUC
subject1,0.877317,0.940619,0.945440,0.936454,0.960864,0.955948,0.936107
subject2,0.811971,0.908399,0.898786,0.872678,0.953166,0.915794,0.893466
subject3,0.701692,0.785243,0.806926,0.780921,0.814678,0.873844,0.793884
subject4,0.881567,0.973993,0.979407,0.940783,0.760006,0.744155,0.879985
subject5,0.798438,0.925332,0.927310,0.879343,0.826613,0.798830,0.859311
subject6,0.831869,0.955862,0.953581,0.911481,0.876658,0.919393,0.908141
subject7,0.836189,0.934141,0.902403,0.928369,0.943793,0.804531,0.891571
subject8,0.744033,0.860204,0.871707,0.813253,0.823746,0.801018,0.818993
subject9,0.841537,0.929823,0.956187,0.976078,0.887596,0.815655,0.901146
subject10,0.963330,0.956401,0.937012,0.924338,0.956502,0.910338,0.941320


# **Scenario 3: Train on All Data, Train on Individual Subjects, Predict for Individual Subjects**

## Scenario 3: Base Model

In [ ]:
LR = LR().double()

In [ ]:
optimizer = torch.optim.Adadelta(LR.parameters(), lr=1, eps=1e-10) # wtf is epsilon???

In [ ]:
for i in range(12):
    print(f'training subject {i+1} out of 12')
    for layer in LR.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    LR.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/Base/All_Subjects_Model"))
    train(LR,subjects[i],events[i],1,2000)
    torch.save(LR.state_dict(), f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/Base/Subject{i+1}_Model")

training subject 1 out of 12


  0%|          | 1/602 [00:00<09:48,  1.02it/s]

	 epoch: 0, iteration: 0/602, loss: 0.0868215380131196


  0%|          | 2/602 [00:01<09:53,  1.01it/s]

	 epoch: 0, iteration: 1/602, loss: 0.09417060085900457


...

	 epoch: 0, iteration: 685/688, loss: 0.07517919526363546


100%|█████████▉| 687/688 [11:28<00:01,  1.00s/it]

	 epoch: 0, iteration: 686/688, loss: 0.13918241344006724


100%|██████████| 688/688 [11:29<00:00,  1.00s/it]

	 epoch: 0, iteration: 687/688, loss: 0.07003871329334702


In [ ]:
allAucs = {}
allPreds = {}
allTests = {}
for i in range(12):
  print(f'predicting for subject {i+1} out of 12')
  LR.load_state_dict(torch.load(f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/Base/Subject{i+1}_Model"))
  preds, test = getPredictions(LR,subjects_test[i],events_test[i],512,2000)
  aucs = [auc(test[:,j], preds[:,j]) for j in range(6)]
  allAucs[f'subject{i+1}'] = aucs
  allPreds[f'subject{i+1}'] = preds
  allTests[f'subject{i+1}'] = test

predicting for subject 1 out of 12
predicting for subject 2 out of 12
predicting for subject 3 out of 12
predicting for subject 4 out of 12
predicting for subject 5 out of 12
predicting for subject 6 out of 12
predicting for subject 7 out of 12
predicting for subject 8 out of 12
predicting for subject 9 out of 12
predicting for subject 10 out of 12
predicting for subject 11 out of 12
predicting for subject 12 out of 12


In [ ]:
# save results

AUC3 = pd.DataFrame.from_dict(allAucs,orient='index')
AUC3['average AUC'] = AUC3.mean(axis=1)
AUC3.from_dict(allAucs,orient='index').to_csv('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/Base/AUC3',index=False)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/Base/allPreds3', 'wb') as f:
    pickle.dump(allPreds, f)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/Base/allTests3', 'wb') as f:
    pickle.dump(allTests, f)

In [ ]:
AUC3

,0,1,2,3,4,5,average AUC
subject1,0.881099,0.971474,0.964731,0.926594,0.952582,0.907466,0.933991
subject2,0.821812,0.896689,0.892154,0.884108,0.954731,0.874611,0.887351
subject3,0.833022,0.861250,0.881240,0.855182,0.857138,0.888930,0.862794
subject4,0.851222,0.946682,0.950419,0.935456,0.813351,0.782962,0.880015
subject5,0.759226,0.964082,0.951525,0.877275,0.862800,0.859548,0.879076
subject6,0.905154,0.959263,0.953833,0.903711,0.832274,0.854958,0.901532
subject7,0.905277,0.962041,0.936206,0.943425,0.848858,0.801965,0.899629
subject8,0.711955,0.832268,0.850154,0.759358,0.899422,0.854716,0.817979
subject9,0.824162,0.930316,0.953152,0.969283,0.923354,0.915703,0.919329
subject10,0.920014,0.960565,0.955286,0.891294,0.950489,0.865527,0.923862


## Scenario 3: CNN Model

In [ ]:
CNN = CNN().double()

In [ ]:
optimizer = torch.optim.Adadelta(CNN.parameters(), lr=1, eps=1e-10) # wtf is epsilon???

In [ ]:
for i in range(12):
    print(f'training subject {i+1} out of 12')
    for layer in CNN.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    CNN.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 2 Models/CNN/All_Subjects_Model"))
    train(CNN,subjects[i],events[i],1,2000)
    torch.save(CNN.state_dict(), f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/CNN/Subject{i+1}_Model")

training subject 10 out of 12


  0%|          | 1/671 [00:05<1:00:54,  5.45s/it]

	 epoch: 0, iteration: 0/671, loss: 0.12116136685531327


  0%|          | 2/671 [00:10<1:00:26,  5.42s/it]

	 epoch: 0, iteration: 1/671, loss: 0.07900663654590098


...

	 epoch: 0, iteration: 685/688, loss: 0.06715018683817155


100%|█████████▉| 687/688 [56:19<00:05,  5.36s/it]

	 epoch: 0, iteration: 686/688, loss: 0.06097306720698028


100%|██████████| 688/688 [56:24<00:00,  4.92s/it]

	 epoch: 0, iteration: 687/688, loss: 0.06056590665153723


In [ ]:
allAucs = {}
allPreds = {}
allTests = {}
for i in range(12):
  print(f'predicting for subject {i+1} out of 12')
  CNN.load_state_dict(torch.load(f"/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/CNN/Subject{i+1}_Model"))
  preds, test = getPredictions(CNN,subjects_test[i],events_test[i],512,2000)
  aucs = [auc(test[:,j], preds[:,j]) for j in range(6)]
  allAucs[f'subject{i+1}'] = aucs
  allPreds[f'subject{i+1}'] = preds
  allTests[f'subject{i+1}'] = test

In [ ]:
# save results

AUC3 = pd.DataFrame.from_dict(allAucs,orient='index')
AUC3['average AUC'] = AUC3.mean(axis=1)
AUC3.from_dict(allAucs,orient='index').to_csv('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/CNN/AUC3',index=False)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/CNN/allPreds3', 'wb') as f:
    pickle.dump(allPreds, f)


with open('/content/drive/My Drive/Colab Notebooks/Bionic AI/Scenario 3 Models/CNN/allTests3', 'wb') as f:
    pickle.dump(allTests, f)

In [ ]:
AUC3

,0,1,2,3,4,5,average AUC
subject1,0.957325,0.994790,0.992333,0.995135,0.981774,0.977459,0.983136
subject2,0.885225,0.943852,0.955470,0.907236,0.964061,0.917130,0.928829
subject3,0.890109,0.935853,0.924668,0.882647,0.900565,0.933439,0.911214
subject4,0.934898,0.980304,0.978643,0.955236,0.868239,0.858608,0.929321
subject5,0.910504,0.956512,0.948588,0.889741,0.940573,0.920686,0.927767
subject6,0.886200,0.967839,0.971828,0.940129,0.914930,0.921833,0.933793
subject7,0.958415,0.975562,0.918291,0.964982,0.971404,0.937764,0.954403
subject8,0.808556,0.919767,0.920348,0.841967,0.955989,0.916666,0.893882
subject9,0.907514,0.939644,0.950168,0.973561,0.855685,0.841432,0.911334
subject10,0.930305,0.962508,0.966722,0.887777,0.970751,0.910997,0.938177
